In [39]:
!pip install -q basicsr==1.4.2 facexlib gfpgan torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install -r requirements.txt
!python setup.py develop
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P weights


Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/dist-packages/setuptools/c

In [40]:

import cv2
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact
from skimage.metrics import structural_similarity as ssim
import numpy as np


Downscale

In [41]:
img= Image.open('/content/images.jpg').convert('RGB')
imgnp = np.array(img)
h , w = imgnp.shape[:2]
print(h,w)
lowres = cv2.resize(imgnp, (w//4, h//4), interpolation=cv2.INTER_AREA)
lowresp = Image.fromarray(lowres)
lowresp.save('/content/lowres.png')


267 189


BICUBIC

In [42]:
bicubic = cv2.resize(lowres, (w, h), interpolation=cv2.INTER_CUBIC)
bicubic = cv2.cvtColor(bicubic, cv2.COLOR_BGR2RGB)
bicubic = Image.fromarray(bicubic)
bicubic.save('/content/bicubic.png')

REAL ESGAN

In [43]:
model = RRDBNet(num_in_ch=3 , num_out_ch=3 ,num_feat= 64, num_block=23, num_grow_ch=32 , scale = 4)
upscaler = RealESRGANer(
    scale=4,
    model_path='/content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth',
    model=model)
sr_img_tuple = upscaler.enhance(np.array(lowresp), outscale=4)
sr_img = sr_img_tuple[0] # Extract the image data from the tuple
sr_img = Image.fromarray(sr_img)
sr_img.save('/content/sr_img.png')

In [44]:
def calculate_ssim(original_image_path, generated_image_path):

    original_img = cv2.imread(original_image_path)
    generated_img = cv2.imread(generated_image_path)


    if original_img.shape != generated_img.shape:

        generated_img = cv2.resize(generated_img, (original_img.shape[1], original_img.shape[0]))

        if original_img.shape[-1] != generated_img.shape[-1]:
             generated_img = cv2.cvtColor(generated_img, cv2.COLOR_BGR2RGB) # Example conversion


    gray_original = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    gray_generated = cv2.cvtColor(generated_img, cv2.COLOR_BGR2GRAY)


    ssim_score, _ = ssim(gray_original, gray_generated, full=True)

    return ssim_score

original_image_path = '/content/images.jpg'  # Assuming the original image is at this path
bicubic_image_path = '/content/bicubic.png'
realesrgan_image_path = '/content/sr_img.png'

ssim_bicubic = calculate_ssim(original_image_path, bicubic_image_path)
ssim_realesrgan = calculate_ssim(original_image_path, realesrgan_image_path)

print(f"SSIM between original and bicubic: {ssim_bicubic:.4f}")
print(f"SSIM between original and Real-ESRGAN: {ssim_realesrgan:.4f}")



SSIM between original and bicubic: 0.7559
SSIM between original and Real-ESRGAN: 0.7102


In [47]:
from google.colab import files
files.download('/content/lowres.png')
files.download('/content/bicubic.png')
files.download('/content/sr_img.png')
files.download('/content/images.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SRCNN